# Case Study 5: Part 2:  Bayesian Estimation

Demand Modeling: 1.202  
Spring 2019  
Alexandra Berke

In [1]:
import numpy as np
import pandas as pd
import random

from mxlMcmc import estimate,test

import h5py

import matplotlib
import matplotlib.pyplot as plt
from math import floor

# Prepare data

In [14]:
# Load data
data = pd.read_csv('swissmetro_long.csv')
# Working with work trips only
data = data[((data['PURPOSE'] != 1) & (data['PURPOSE'] != 3)) != True]
print("shape of data:", data.shape)
data.head()

shape of data: (19143, 20)


,AGE,AV,CO,DEST,FIRST,GA,HE,ID,INCOME,LUGGAGE,MALE,ORIGIN,PURPOSE,SEATS,TICKET,TT,WHO,altID,chosen,obsID
0,3,1,48,1,0,0,120.0,1,2,0,0,2,1,0.0,1,112,1,1,0,0
1,3,1,52,1,0,0,20.0,1,2,0,0,2,1,0.0,1,63,1,2,1,0
2,3,1,65,1,0,0,0.0,1,2,0,0,2,1,0.0,1,117,1,3,0,0
3,3,1,48,1,0,0,30.0,1,2,0,0,2,1,0.0,1,103,1,1,0,1
4,3,1,49,1,0,0,10.0,1,2,0,0,2,1,0.0,1,60,1,2,1,1


Divide the data into training (first 8 choices) and test (last choice)

In [15]:
testdata = data[(data['obsID']+1) % 9 ==0]
testdata.shape
data = data[(data['obsID']+1) % 9 > 0]
print("shape after dividing test data (8/9) vs training data (1/8):", data.shape)

shape after dividing test data (8/9) vs training data (1/8): (17016, 20)


Prepare the _training_ data and variables

In [19]:
indID = np.array(data['ID'].values, dtype = 'int64').reshape((-1,1))
obsID = np.array(data['obsID'].values, dtype = 'int64').reshape((-1,1))
altID = np.array(data['altID'].values, dtype = 'int64').reshape((-1,1))

chosen = np.array(data['chosen'].values, dtype = 'int64')

tt = np.array(data['TT'].values, dtype = 'float64').reshape((-1,1)) / 1
cost = np.array(data['CO'].values, dtype = 'float64').reshape((-1,1)) / 1
he = np.array(data['HE'].values, dtype = 'float64').reshape((-1,1)) / 1
ga = np.array(data['GA'].values, dtype = 'int64').reshape((-1,1))
cost[(altID[:,0] <= 2) & (ga[:,0] == 1),0] = 0

const2 = 1 * (altID == 2)
const3 = 1 * (altID == 3)

Prepare the _test_ data and variables (just once!)

In [21]:
testindID = np.array(testdata['ID'].values, dtype = 'int64').reshape((-1,1))
testobsID = np.array(testdata['obsID'].values, dtype = 'int64').reshape((-1,1))
testaltID = np.array(testdata['altID'].values, dtype = 'int64').reshape((-1,1))

testchosen = np.array(testdata['chosen'].values, dtype = 'int64')

testtt = np.array(testdata['TT'].values, dtype = 'float64').reshape((-1,1)) / 1
testcost = np.array(testdata['CO'].values, dtype = 'float64').reshape((-1,1)) / 1
testhe = np.array(testdata['HE'].values, dtype = 'float64').reshape((-1,1)) / 1
testga = np.array(testdata['GA'].values, dtype = 'int64').reshape((-1,1))
testcost[(testaltID[:,0] <= 2) & (testga[:,0] == 1),0] = 0

testconst2 = 1 * (testaltID == 2)
testconst3 = 1 * (testaltID == 3)

# 0. (MNL) Multinomial logit

This model is the same as specified in the handout.

- V_car = ASC_car + BETA_time * TIME_car + BETA_cost * COST_car
- V_SM = ASC_SM + BETA_time * TIME_SM + BETA_cost * COST_SM
- V_train = BETA_time*TIME_train + BETA_cost * COST_train

In [17]:
xFix = np.hstack((const2, const3, cost / 100, tt / 100))
xRnd = np.zeros((0, 0))

#Fixed parameter distributions
#0: normal
#1: log-normal (to assure that fixed parameter is striclty negative or positive)
xFix_trans = np.array([0, 0, 0, 0])

#Random parameter distributions
#0: normal
#1: log-normal
#2: S_B
xRnd_trans = np.array([0, 0, 0])

paramFix_inits = np.zeros((xFix.shape[1],))
zeta_inits = np.zeros((xRnd.shape[1],))
Omega_inits = 0.1 * np.eye(xRnd.shape[1])

A = 1.04
nu = 2
diagCov = True

mcmc_nChain = 2
mcmc_iterBurn = 20000
mcmc_iterSample = 20000
mcmc_thin = 5
mcmc_iterMem = 20000
mcmc_disp = 1000
seed = 4711
simDraws = 10000

rho = 0.1
rhoF = 0.01

modelName = 'model'
deleteDraws = False

results = estimate(
        mcmc_nChain, mcmc_iterBurn, mcmc_iterSample, mcmc_thin, mcmc_iterMem, mcmc_disp, 
        seed, simDraws,
        rhoF, rho,
        modelName, deleteDraws,
        A, nu, diagCov,
        paramFix_inits, zeta_inits, Omega_inits,
        indID, obsID, altID, chosen,
        xFix, xRnd,
        xFix_trans, xRnd_trans)

 
Computation time [s]: 13.232194900512695
 
Fixed parameters:
       mean  std. dev.      2.5%     97.5%      Rhat
0  0.766398   0.058094  0.651846  0.881184  1.006574
1  0.602931   0.048624  0.507730  0.699931  1.009486
2 -1.067394   0.054099 -1.175383 -0.962256  1.007610
3 -1.261752   0.059792 -1.382350 -1.148355  1.006176
 
Log-likelihood (simulated at posterior means): -4683.345666756498


Test

In [23]:
testxFix = np.hstack((testconst2, testconst3, testcost / 100, testtt / 100))
testxRnd = np.zeros((0, 0))

test(results,
        seed, simDraws, testindID, testobsID, testaltID, testchosen,
        testxFix, testxRnd,
        xFix_trans, xRnd_trans)

 
Log-likelihood (simulated at posterior means): -649.1198278087231


{'unconditionalLogLik': -649.1198278087231,
 'conditionalLogLik': -649.1198278087231,
 'unconditionalChosenProb': 0.4974726597051809,
 'conditionalChosenProb': 0.4974726597051809}

# 1. (MXL 1) 
This is a logit mixture model where cost and time are normally distributed random variables, with ASCs.
This model is the same as that specified in Part 2 of the handout.

- V_car = ASC_car + BETA_time * TIME_car + BETA_cost * COST_car
- V_SM = ASC_SM + BETA_time * TIME_SM + BETA_cost * COST_SM
- V_train = BETA_time*TIME_train + BETA_cost * COST_train

Where BETA_time and BETA_cost are normally distributed random variables.

In [24]:
xFix = np.hstack((const2, const3))
xRnd = np.hstack((cost / 10, tt / 10)) 


#Fixed parameter distributions
#0: sign unrestricted
#1: striclty negative or positive
xFix_trans = np.array([0, 0])

#Random parameter distributions
#0: normal
#1: log-normal
#2: S_B
xRnd_trans = np.array([0, 0])   # if you set these to 1, you obtain the log-normal distribution.

paramFix_inits = np.zeros((xFix.shape[1],))
zeta_inits = np.zeros((xRnd.shape[1],))
Omega_inits = 0.1 * np.eye(xRnd.shape[1])

A = 1.04
nu = 2
diagCov = True

mcmc_nChain = 2
mcmc_iterBurn = 20000
mcmc_iterSample = 20000
mcmc_thin = 5
mcmc_iterMem = 20000
mcmc_disp = 1000
seed = 4711
simDraws = 10000

rho = 0.1
rhoF = 0.01

modelName = 'model'
deleteDraws = False

results = estimate(
        mcmc_nChain, mcmc_iterBurn, mcmc_iterSample, mcmc_thin, mcmc_iterMem, mcmc_disp, 
        seed, simDraws,
        rhoF, rho,
        modelName, deleteDraws,
        A, nu, diagCov,
        paramFix_inits, zeta_inits, Omega_inits,
        indID, obsID, altID, chosen,
       xFix, xRnd,
        xFix_trans, xRnd_trans)

 
Computation time [s]: 80.26553225517273
 
Fixed parameters:
       mean  std. dev.      2.5%     97.5%      Rhat
0  0.403494   0.104516  0.199164  0.611025  1.009599
1  0.832593   0.089524  0.659136  1.006793  1.013095
 
Random parameters (means):
       mean  std. dev.      2.5%     97.5%      Rhat
0 -0.406494   0.028875 -0.464803 -0.351088  1.008459
1 -0.485257   0.026475 -0.537479 -0.434834  1.010140
 
Random parameters (standard deviations):
       mean  std. dev.      2.5%     97.5%      Rhat
0  0.468416   0.031896  0.410759  0.534444  1.009343
1  0.451258   0.023075  0.407650  0.497703  1.017810
 
Random parameters (covariance matrix):
       mean  std. dev.      2.5%     97.5%      Rhat
0  0.220431   0.030116  0.168723  0.285631  1.009162
1  0.000000   0.000000  0.000000  0.000000       NaN
2  0.000000   0.000000  0.000000  0.000000       NaN
3  0.204166   0.020923  0.166179  0.247708  1.018958
 
Random parameters (correlation matrix):
   mean  std. dev.  2.5%  97.5%  Rhat
0  

A problem with using normally distributed random variables for the cost and time coefficients is that while we understand that these coefficients should always be negative, they are not guaranteed to be negative.  
The probability that they have positive values can be computed from their estimated mean and standard deviation values.  The number of individual  specific coefficients with values greater than zero can also be counted directly from the posterior estimates obtained from the Gibbs sampler.

The positive individual specific coefficients for cost and time are counted below

In [25]:
# Count individual specific coefficient values > 0 for cost and time
# Cost: item 0 in results
# Time: item 1 in results
cost_coeffs_gt_zero_count = 0
time_coeffs_gt_zero_count = 0

results_df = pd.DataFrame(results['postMean_paramRnd'])
for index, row in results_df.iterrows():
    if (row[0] > 0):
        cost_coeffs_gt_zero_count += 1
    if (row[1] > 0):
        time_coeffs_gt_zero_count += 1

# How many??
total_coeffs = results_df.shape[0] # 752
print("Total estimated individual specific coefficients:", total_coeffs)
print("# estimated individual specific COST coefficients > 0: ", cost_coeffs_gt_zero_count, " Fraction: ",  cost_coeffs_gt_zero_count/total_coeffs)
print("# estimated individual specific TIME coefficients > 0: ", time_coeffs_gt_zero_count, " Fraction: ",  time_coeffs_gt_zero_count/total_coeffs)

# estimated individual specific COST coefficients > 0:  90  Fraction:  0.1196808510638298
Total estimated individual specific coefficients: 752
# estimated individual specific TIME coefficients > 0:  89  Fraction:  0.11835106382978723


Predictions on the test data

In [28]:
testxFix = np.hstack((testconst2, testconst3))
testxRnd = np.hstack((testcost / 10, testtt / 10))

test(results,
        seed, simDraws, testindID, testobsID, testaltID, testchosen,
        testxFix, testxRnd,
        xFix_trans, xRnd_trans)

 
Log-likelihood (simulated at posterior means): -651.9912237048596


{'unconditionalLogLik': -651.9912237048596,
 'conditionalLogLik': -481.20318558221015,
 'unconditionalChosenProb': 0.5118410425851324,
 'conditionalChosenProb': 0.6945433440830614}

# 2. (MXL 2) 
This is a logit mixture model where *all* variables: cost, time, and alternative specific parameters are *normally* distributed random variables.

- V_car = ASC_car + BETA_time * TIME_car + BETA_cost * COST_car
- V_SM = ASC_SM + BETA_time * TIME_SM + BETA_cost * COST_SM
- V_train = BETA_time*TIME_train + BETA_cost * COST_train

Where BETA_time, BETA_cost, ASC_car,and ASC_SM are normally distributed random variables.

Note that in this model specification, both the mean and variance for the train alternative are fixed to zero in order to normalize the model.  Proper model estimation would be done in two stages.  The first stage would estimate the unidentified model without the train variance fixed to zero in order to find the smallest variance value.  The smallest value would then be normalized to zero for the  second stage of estimation.


In [28]:
xFix = np.empty([0,0]) #np.hstack(())
xRnd = np.hstack((const2, const3, cost / 10, tt / 10))

#Fixed parameter distributions
#0: sign unrestricted
#1: striclty negative or positive
xFix_trans = np.array([])

#Random parameter distributions
#0: normal
#1: log-normal
#2: S_B
xRnd_trans = np.array([0, 0, 0, 0])   # if you set these to 1, you obtain the log-normal distribution.

paramFix_inits = np.zeros((xFix.shape[1],))
zeta_inits = np.zeros((xRnd.shape[1],))
Omega_inits = 0.1 * np.eye(xRnd.shape[1])

A = 1.04
nu = 2
diagCov = True

mcmc_nChain = 2
# Note the extra iterations
mcmc_iterBurn = 200000  # vs: 20000
mcmc_iterSample = 200000  # vs: 20000
mcmc_thin = 5
mcmc_iterMem = 20000
mcmc_disp = 1000
seed = 4711
simDraws = 10000

rho = 0.1
rhoF = 0.01

modelName = 'model'
deleteDraws = False

results = estimate(
        mcmc_nChain, mcmc_iterBurn, mcmc_iterSample, mcmc_thin, mcmc_iterMem, mcmc_disp, 
        seed, simDraws,
        rhoF, rho,
        modelName, deleteDraws,
        A, nu, diagCov,
        paramFix_inits, zeta_inits, Omega_inits,
        indID, obsID, altID, chosen,
       xFix, xRnd,
        xFix_trans, xRnd_trans)

 
Computation time [s]: 556.8028385639191
 
Random parameters (means):
       mean  std. dev.      2.5%     97.5%      Rhat
0 -0.006811   0.192645 -0.380114  0.375000  1.001087
1  0.700808   0.228102  0.254882  1.151687  1.000768
2 -0.564432   0.037921 -0.641728 -0.493622  1.028296
3 -0.715819   0.037580 -0.791738 -0.644111  1.028836
 
Random parameters (standard deviations):
       mean  std. dev.      2.5%     97.5%      Rhat
0  2.227488   0.215013  1.818990  2.661734  1.003090
1  3.603926   0.269848  3.098442  4.152504  1.001957
2  0.387283   0.041217  0.309406  0.471667  1.081652
3  0.469561   0.032644  0.407820  0.536623  1.127215
 
Random parameters (covariance matrix):
         mean  std. dev.      2.5%      97.5%      Rhat
0    5.007933   0.966097  3.308726   7.084830  1.006706
1    0.000000   0.000000  0.000000   0.000000       NaN
2    0.000000   0.000000  0.000000   0.000000       NaN
3    0.000000   0.000000  0.000000   0.000000       NaN
4    0.000000   0.000000  0.000000 

The number of positive individual specific coefficients for cost and time are counted below.

In [30]:
# Count individual specific coefficient values > 0 for cost and time
# Cost: item 0 in results
# Time: item 1 in results
cost_coeffs_gt_zero_count = 0
time_coeffs_gt_zero_count = 0

results_df = pd.DataFrame(results['postMean_paramRnd'])
for index, row in results_df.iterrows():
    if (row[2] > 0):
        cost_coeffs_gt_zero_count += 1
    if (row[3] > 0):
        time_coeffs_gt_zero_count += 1

# How many??
total_coeffs = results_df.shape[0] # 752
print("Total estimated individual specific coefficients:", total_coeffs)
print("# estimated individual specific COST coefficients > 0: ", cost_coeffs_gt_zero_count, " Fraction: ",  cost_coeffs_gt_zero_count/total_coeffs)
print("# estimated individual specific TIME coefficients > 0: ", time_coeffs_gt_zero_count, " Fraction: ",  time_coeffs_gt_zero_count/total_coeffs)

Total estimated individual specific coefficients: 752
# estimated individual specific COST coefficients > 0:  8  Fraction:  0.010638297872340425
# estimated individual specific TIME coefficients > 0:  26  Fraction:  0.034574468085106384


Predictions on the test data

In [44]:
testxFix = np.empty([0,0])
testxRnd = np.hstack((testconst2, testconst3, testcost / 10, testtt / 10))

test(results,
        seed, simDraws, testindID, testobsID, testaltID, testchosen,
        testxFix, testxRnd,
        xFix_trans, xRnd_trans)

 
Log-likelihood (simulated at posterior means): -647.8026517686915


{'unconditionalLogLik': -647.8026517686915,
 'conditionalLogLik': -455.77582816623067,
 'unconditionalChosenProb': 0.5195688295308631,
 'conditionalChosenProb': 0.7436788911173949}

# 3. (MXL 3) 
This is a logit mixture model where cost and time are log-normally distributed random variables, and the alternative specific parameters are constants (ASC’s).

- V_car = ASC_car + BETA_time * TIME_car + BETA_cost * COST_car
- V_SM = ASC_SM + BETA_time * TIME_SM + BETA_cost * COST_SM
- V_train = BETA_time*TIME_train + BETA_cost * COST_train

Where BETA_time and BETA_cost are log-normally distributed random variables, and ASC_car and ASC_SM alternative specific constants (the alternative specific constant for the train alternative is normalized to zero).

In [46]:
xFix = np.hstack((const2, const3))
xRnd = np.hstack((-cost / 10, -tt / 10)) 


#Fixed parameter distributions
#0: sign unrestricted
#1: striclty negative or positive
xFix_trans = np.array([0, 0])

#Random parameter distributions
#0: normal
#1: log-normal
#2: S_B
xRnd_trans = np.array([1, 1])   # if you set these to 1, you obtain the log-normal distribution.

paramFix_inits = np.zeros((xFix.shape[1],))
zeta_inits = np.zeros((xRnd.shape[1],))
Omega_inits = 0.1 * np.eye(xRnd.shape[1])

A = 1.04
nu = 2
diagCov = True

mcmc_nChain = 2
mcmc_iterBurn = 20000
mcmc_iterSample = 20000
mcmc_thin = 5
mcmc_iterMem = 20000
mcmc_disp = 1000
seed = 4711
simDraws = 10000

rho = 0.1
rhoF = 0.01

modelName = 'model'
deleteDraws = False

results = estimate(
        mcmc_nChain, mcmc_iterBurn, mcmc_iterSample, mcmc_thin, mcmc_iterMem, mcmc_disp, 
        seed, simDraws,
        rhoF, rho,
        modelName, deleteDraws,
        A, nu, diagCov,
        paramFix_inits, zeta_inits, Omega_inits,
        indID, obsID, altID, chosen,
       xFix, xRnd,
        xFix_trans, xRnd_trans)

 
Computation time [s]: 83.12688279151917
 
Fixed parameters:
       mean  std. dev.      2.5%     97.5%      Rhat
0 -0.230958   0.079589 -0.387074 -0.076887  1.008193
1  0.562897   0.071738  0.422578  0.706022  1.010078
 
Random parameters (means):
       mean  std. dev.      2.5%     97.5%      Rhat
0 -1.643367   0.125988 -1.898574 -1.398229  1.002418
1 -0.841716   0.068357 -0.973698 -0.705896  1.008828
 
Random parameters (standard deviations):
       mean  std. dev.      2.5%     97.5%      Rhat
0  1.747499   0.123857  1.516690  1.998678  1.000737
1  1.355614   0.072767  1.222574  1.503236  1.002607
 
Random parameters (covariance matrix):
       mean  std. dev.      2.5%     97.5%      Rhat
0  3.069095   0.437016  2.300350  3.994712  1.000563
1  0.000000   0.000000  0.000000  0.000000       NaN
2  0.000000   0.000000  0.000000  0.000000       NaN
3  1.842986   0.198975  1.494687  2.259717  1.003668
 
Random parameters (correlation matrix):
   mean  std. dev.  2.5%  97.5%  Rhat
0  

Predictions on the test data

In [47]:
testxFix = np.hstack((testconst2, testconst3))
testxRnd = np.hstack((-testcost / 10, -testtt / 10))

test(results,
        seed, simDraws, testindID, testobsID, testaltID, testchosen,
        testxFix, testxRnd,
        xFix_trans, xRnd_trans)

 
Log-likelihood (simulated at posterior means): -659.1062774621745


{'unconditionalLogLik': -659.1062774621745,
 'conditionalLogLik': -572.9883553544789,
 'unconditionalChosenProb': 0.504376263015899,
 'conditionalChosenProb': 0.6592616482282548}